In [15]:
# code to upgrade dataset
!pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12

In [ ]:
from datasets import load_dataset
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
import numpy as np
import evaluate

dataset = load_dataset("google-research-datasets/go_emotions", "simplified")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/350k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/347k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/43410 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5426 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5427 [00:00<?, ? examples/s]

In [ ]:
# Load tokenizer
model_name = "lishaangral/roberta-mental-health"
tokenizer = RobertaTokenizer.from_pretrained(model_name)

# Get the number of labels from the dataset features
num_labels = dataset['train'].features['labels'].feature.num_classes

# Preprocessing: tokenize and encode labels
import numpy as np

def preprocess(example):
    encoding = tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)
    labels = np.zeros(num_labels, dtype=np.float32)  # float32 for BCE loss
    for idx in example["labels"]:
        if 0 <= idx < num_labels:
            labels[idx] = 1.0
    encoding["labels"] = labels
    return encoding

encoded_dataset = dataset.map(preprocess, batched=False)
encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

In [3]:
def transform(example):
    return {
        "input_ids": torch.tensor(example["input_ids"], dtype=torch.long),
        "attention_mask": torch.tensor(example["attention_mask"], dtype=torch.long),
        "labels": torch.tensor(example["labels"], dtype=torch.float32),
    }

encoded_dataset.set_transform(transform)

In [4]:
print(encoded_dataset["train"][0]["labels"].dtype)


torch.float32


In [5]:
# Model: Multi-label classification
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, problem_type="multi_label_classification", ignore_mismatched_sizes=True)

# Compute metrics (per-label accuracy + f1/precision/recall)
def compute_metrics(pred):
    logits, labels = pred
    probs = torch.sigmoid(torch.tensor(logits))
    preds = (probs > 0.5).int().numpy()
    labels = labels.astype(int)

    f1 = f1_score(labels, preds, average="micro")
    precision = precision_score(labels, preds, average="micro")
    recall = recall_score(labels, preds, average="micro")

    # Per-label accuracy
    per_label_accuracy = (preds == labels).mean()

    return {
        "f1_micro": f1,
        "precision_micro": precision,
        "recall_micro": recall,
        "per_label_accuracy": per_label_accuracy
    }

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1_micro",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train
trainer.train()

# Evaluate
trainer.evaluate()

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at lishaangral/roberta-mental-health and are newly initialized because the shapes did not match:
- classifier.out_proj.bias: found shape torch.Size([7]) in the checkpoint and torch.Size([28]) in the model instantiated
- classifier.out_proj.weight: found shape torch.Size([7, 768]) in the checkpoint and torch.Size([28, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-5-120320298.py:40: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lishaangral (lishaangral-netaji-subhas-university-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,F1 Micro,Precision Micro,Recall Micro,Per Label Accuracy
1,0.097500,0.091029,0.536463,0.745054,0.419122,0.969584
2,0.083000,0.084519,0.563267,0.725074,0.460502,0.970012
3,0.073500,0.083693,0.579718,0.709392,0.490125,0.970157


{'eval_loss': 0.08369316160678864,
 'eval_f1_micro': 0.5797182054134223,
 'eval_precision_micro': 0.7093920145190563,
 'eval_recall_micro': 0.4901253918495298,
 'eval_per_label_accuracy': 0.9701569164341004,
 'eval_runtime': 35.1624,
 'eval_samples_per_second': 154.312,
 'eval_steps_per_second': 9.669,
 'epoch': 3.0}

In [6]:
# Save the tuned model
tokenizer.save_pretrained("./results")
model.save_pretrained("./results")

In [7]:
from huggingface_hub import notebook_login
notebook_login()

In [8]:
# Load your fine-tuned model
model = RobertaForSequenceClassification.from_pretrained("./results")
tokenizer = RobertaTokenizer.from_pretrained("./results")

# Define repo name
repo_name = "lishaangral/roberta-mental-health-v2"

# Push model and tokenizer to Hugging Face Hub
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/lishaangral/roberta-mental-health-v2/commit/eb4433cd1ecb410b5463882e0fcf6bfdcaafa122', commit_message='Upload tokenizer', commit_description='', oid='eb4433cd1ecb410b5463882e0fcf6bfdcaafa122', pr_url=None, repo_url=RepoUrl('https://huggingface.co/lishaangral/roberta-mental-health-v2', endpoint='https://huggingface.co', repo_type='model', repo_id='lishaangral/roberta-mental-health-v2'), pr_revision=None, pr_num=None)